In [ ]:
!wget -O video_frames_long.zip https://www.dropbox.com/s/wq4f8m959yloai3/video_frames_long.zip?dl=1

In [5]:
!pip install torch

     |████████████████████████████▋   | 673.4 MB 75.0 MB/s eta 0:00:02     |████████████▎                   | 287.8 MB 62.5 MB/s eta 0:00:08     |███████████████████████▊        | 556.5 MB 78.4 MB/s eta 0:00:03     |██████████████████████████      | 609.1 MB 78.4 MB/s eta 0:00:02

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



     |████████████████████████████████| 752.0 MB 50.3 MB/s eta 0:00:01     |████████████████████████████████| 752.0 MB 832 bytes/s 


In [1]:
import cv2
import os
import numpy as np
import shutil
import torch
import zipfile
import cv2
import os
import numpy as np

In [7]:
with zipfile.ZipFile("video_frames_long.zip", 'r') as zip_ref:
    zip_ref.extractall('video_frames_long')

In [ ]:
import cv2
import os
import numpy as np
not_pres = []
def video_frames(video_folder, new_folder, snap_full = True):
    videos = os.listdir(path = video_folder) 
    os.mkdir(new_folder)
    for elem in videos:
        cap = cv2.VideoCapture(video_folder+elem)
        fps = int(cap.get(5))
        lp = np.linspace(33, fps * 10 , num = 20)
        lp = lp.astype('int')
        frames = cap.get(cv2.CAP_PROP_FRAME_COUNT)
        frames = int(frames)
        if snap_full == True:  
            lp = np.linspace(33, frames-4, num = 20)
            lp = lp.astype('int')
        os.mkdir(new_folder + elem.split('.')[0])
        for ind in lp:
            cap.set(cv2.CAP_PROP_POS_FRAMES, ind)
            ret, frame = cap.read()
            if ret == False:
                print('hello')
                print(elem)
                not_pres.append(elem)
            else:
                cv2.imwrite(new_folder+elem.split('.')[0] + '/'+ str(ind) + '.png', frame)

In [ ]:
video_frames('videos11_k/videos_11k', 'video_frames_short/', snap_full = False)

In [ ]:
len(set(not_pres))

In [ ]:
not_pres = list(set(not_pres))

In [ ]:
from pathlib import Path
import shutil
for elem in not_pres:
    adr = elem.split('.')[0]
    dirpath = Path('video_frames_normal', adr)
    if dirpath.exists() and dirpath.is_dir():
        shutil.rmtree(dirpath)

In [2]:
import pandas as pd
import cv2
import os
import numpy as np
import shutil
import torch
import zipfile
import cv2
import os
import numpy as np
dataset = pd.read_csv("dataset.csv")

In [3]:
dataset['video_names'] = ''
i = 0
for elem in dataset['video']:
    dataset['video_names'][i] = elem.split('.')[0]
    i+=1

/anaconda/envs/py37_default/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


In [4]:
elems = os.listdir('video_frames_long/video_frames_long')

In [5]:
dataset = dataset[dataset['video_names'].isin(elems)]

In [6]:
dataset = dataset.reset_index(drop = True)

In [7]:
import datetime as dt
dataset['published_at'] = pd.to_datetime(dataset['published_at'])
diff = dt.datetime.now() - dataset['published_at']
dataset['published_at'] = diff.dt.total_seconds()

In [8]:
dataset = dataset.sort_values(by = 'published_at',   ascending=False)

In [9]:
dataset = dataset.reset_index(drop = True)

In [10]:
train_ids = int(0.8*len(dataset))
val_ids = int(0.8*len(dataset)) + int(0.1*len(dataset))

In [11]:
dataset['popular'] = 0.0
dataset['popular'][dataset.view_count_new.gt(dataset.view_count_new.quantile(0.8))] = 1.0

/anaconda/envs/py37_default/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [19]:
train = dataset[:train_ids]
val = dataset[train_ids:val_ids]
test = dataset[val_ids:len(dataset)]

In [20]:
sum(train['popular'] == 1)

1743

In [21]:
sum(val['popular'] == 1)

197

In [22]:
sum(test['popular'] == 1)

130

In [23]:
len(train)

8278

In [24]:
len(val)

1034

In [25]:
len(test)

1036

In [26]:
vids_train = train['video_id']
y_train = train['popular']

vids_val = val['video_id']
y_val = val['popular']

vids_test = test['video_id']
y_test = test['popular']

In [28]:
!pip install torchvision

     |████████████████████████████████| 6.6 MB 20.7 MB/s eta 0:00:01


In [27]:
from torchvision import transforms
transform_train = transforms.Compose([transforms.Resize([196, 196]),
                                transforms.ToTensor(),
                                transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])])

transform_test = transforms.Compose([transforms.Resize([196, 196]),  transforms.ToTensor(),transforms.Normalize(mean=[0.485, 0.456, 0.406], 
                                std=[0.229, 0.224, 0.225])])



In [47]:
from torch.utils import data
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision.models as models
import torchvision.transforms as transforms
from PIL import Image
class Video_LRCN_dataset(data.Dataset):
    def __init__(self, data_path, frame_folders,  y,  transform=None):
        "Initialization"
        self.data_path = data_path
        self.frame_folders = frame_folders
        self.transform = transform
        self.y = y
    
    def __len__(self):
        return len(self.y)
    
    def read_images_folder(self, folder):
        frames_vid = os.listdir(os.path.join(self.data_path, folder))
        def myFunc(e):
            return int(e.split('.')[0])
        out = []
        frames_vid.sort(reverse=False, key=myFunc)
        for pic in frames_vid:
            image = Image.open(os.path.join(self.data_path, folder, pic))
            if self.transform:
                image = self.transform(image)
            out.append(image)
        out = torch.stack(out, dim = 0)
        return out
    
    def __getitem__(self, index):
        folder = self.frame_folders[index]
        out = self.read_images_folder(folder)
        y = list(self.y)[index]
        return out, y

In [48]:
dataset_train = Video_LRCN_dataset('video_frames_long/video_frames_long',list(vids_train), list(y_train),  transform_train)
dataset_val = Video_LRCN_dataset('video_frames_long/video_frames_long', list(vids_val), list(y_val),  transform_test)

In [49]:
from torch.utils import data
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision.models as models
import torchvision.transforms as transforms
from PIL import Image 


In [56]:
class PopLRCNCAT(nn.Module):
    def __init__(self):
        super(PopLRCNCAT, self).__init__()
        resnet = models.resnet152(pretrained=True)
        in_feats = resnet.fc.in_features
        base_mod = list(resnet.children())[:-1]
        self.base_mod = nn.Sequential(*base_mod)
        for child in self.base_mod.children():
            for param in child.parameters():
                param.requires_grad = False
        self.out1 = nn.Linear(in_feats, 512)
        self.func1 = nn.LeakyReLU(0.2)
        self.bn1 = nn.BatchNorm1d(512)
        self.out2 = nn.Linear(512, 512)
        self.func2 = nn.LeakyReLU(0.2)
        self.bn2 = nn.BatchNorm1d(512)
        self.out3 = nn.Linear(512, 256)
        self.fc1 = nn.Linear(2560, 1024)
        self.func1 = nn.LeakyReLU(0.2)
        self.fc2 = nn.Linear(1024, 256)
        self.func2 = nn.LeakyReLU(0.2)
        self.dropout2 = nn.Dropout(0.3)
        self.fc3 = nn.Linear(256, 64)
        self.func3 = nn.LeakyReLU(0.2)
        self.dropout3 = nn.Dropout(0.3)
        self.fc4 = nn.Linear(64, 1)
        
    
    def forward(self, x):
        x_embed = []
        for i in range(x.size(1)):
            x_im = self.base_mod(x[:,i,:,:,:])
            x_im = x_im.view(x_im.size(0), -1)
            x_im = self.out1(x_im)
            x_im = self.func1(x_im)
            x_im = self.bn1(x_im)
            x_im = self.out2(x_im)
            x_im = self.func2(x_im)
            x_im = self.bn2(x_im)
            x_im = self.out3(x_im)
            x_embed.append(x_im)
        x_embed = torch.cat(x_embed, dim=-1)
        out = self.fc1(x_embed)
        out = self.func1(out)
        out = self.fc2(out)
        out = self.func2(out)
        out = self.dropout2(out)
        out = self.fc3(out)
        out = self.func3(out)
        out = self.dropout3(out)
        out = self.fc4(out)
        return out



In [57]:
mod = PopLRCNCAT().cuda()
model = nn.DataParallel(mod)

In [58]:
train_dl = data.DataLoader(dataset_train, batch_size = 120)
val_dl = data.DataLoader(dataset_val, batch_size = 32)

In [59]:
model = nn.DataParallel(mod)

In [35]:
def plot_history(train_history, val_history, title='loss'):
    plt.figure()
    plt.plot(range(len(train_history)), train_history, label = 'train')
    x,y = zip(*val_history)
    plt.plot(x, y, ms = 10,  label = 'val', color = 'y',  marker = '+', ls = '')
    plt.xlabel('train steps')
    plt.title(title)
    plt.legend(loc = 'upper right')
    plt.show()


In [44]:
def binary_acc(y_pred, y_test):
    y_pred_tag = torch.round(torch.sigmoid(y_pred))
    y_pred_tag = y_pred_tag.squeeze(dim = -1)
    correct_results_sum = (y_pred_tag == y_test).sum().float()
    acc = correct_results_sum/y_test.shape[0]
    acc = acc.item()
    return acc

In [60]:
from sklearn.metrics import roc_auc_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from torch.utils.tensorboard import SummaryWriter
train_log = []
val_log = []
import matplotlib.pyplot as plt
def train_model(model, epoch, lr):
    parameters = filter(lambda p: p.requires_grad, model.parameters())
    optimizer = torch.optim.Adam(parameters, lr=lr)
    opt_scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode = 'max', patience =  1, factor=0.1)
    criterion = nn.BCEWithLogitsLoss()
    writer = SummaryWriter(comment = 'LRCNconcat2')
    for i in range(epoch):
        model.train()
        sum_loss = 0.0
        total = 0
        train_log_epoch = []
        train_val_epoch = []
        num_batch = 0
        total_predicts = []
        total_corrects = []
        smth = 0
        for elem in train_dl:
            optimizer.zero_grad()
            frames = elem[0].float().cuda()
            y = elem[1].float().cuda()
            y_pred = model(frames)
            smth += 1
            print(y.shape)
            print(y_pred.shape)
            loss = criterion(y_pred, y.unsqueeze(-1))
            loss.backward()
            optimizer.step()
            loss = loss.item()
            acc = binary_acc(y_pred, y)
            train_log_epoch.append(acc)
            y_pred_sig = torch.sigmoid(y_pred)
            y_pred_sig = y_pred_sig.flatten().tolist()

            total_predicts.extend(y_pred_sig)
            total_corrects.extend(y.flatten().tolist())
        
        acc = np.mean(train_log_epoch)
        binary_preds = np.array(total_predicts)>0.5
        binary_preds = binary_preds.astype(int)
        binary_preds = list(binary_preds)
        print('train')
        prec = precision_score(total_corrects, binary_preds)
        rec = recall_score(total_corrects, binary_preds)
        roc_auc = roc_auc_score(total_corrects, total_predicts)
        writer.add_scalar('precision_train', prec, i)
        writer.add_scalar('recall_train', rec, i)
        writer.add_scalar('roc_auc_train', roc_auc, i)
        writer.add_scalar('accuracy_train', acc, i)
        print(prec)
        print(rec)
        print(roc_auc)
        print(acc)
        prec_test, rec_test, roc_auc_test, acc_test = val(model, val_dl)
        opt_scheduler.step(acc_test)
        writer.add_scalar('precision_test', prec_test, i)
        writer.add_scalar('recall_test', rec_test, i)
        writer.add_scalar('roc_auc_test', roc_auc_test, i)
        writer.add_scalar('accuracy_test', acc_test, i)
        torch.save(model.state_dict(), 'checkpoint/model_concat'+str(i)+'.pth')
        print(prec_test)
        print(rec_test)
        print(roc_auc_test)
        print(acc_test)
        writer.close()
    return model


def val(model, valid_dl):
    model.eval()
    sum_loss = 0.0
    val_epoch = []
    total_preds = []
    total_corrects = []
    criterion = nn.BCEWithLogitsLoss()
    for elem in valid_dl:
        frames = elem[0].cuda()
        y = elem[1].float().cuda()
        y_pred = model(frames)
        y_pred_sig = torch.sigmoid(y_pred)
        y_pred_sig = y_pred_sig.flatten().tolist()
        total_preds.extend(y_pred_sig)
        total_corrects.extend(y.flatten().tolist())
        acc = binary_acc(y_pred, y)
        val_epoch.append(acc)

    binary_preds = np.array(total_preds)>0.5
    binary_preds = binary_preds.astype(int)
    binary_preds = list(binary_preds)
    print('test')
    print(total_corrects)
    print(binary_preds)
    print(total_preds)
    prec = precision_score(total_corrects, binary_preds)
    rec = recall_score(total_corrects, binary_preds)
    roc_auc = roc_auc_score(total_corrects, total_preds)
    acc = np.mean(val_epoch)
    return prec, rec, roc_auc, acc

In [62]:
model = train_model(model, 30, 0.001)

torch.Size([120])
torch.Size([120, 1])
torch.Size([120])
torch.Size([120, 1])
torch.Size([120])
torch.Size([120, 1])
torch.Size([120])
torch.Size([120, 1])
torch.Size([120])
torch.Size([120, 1])
torch.Size([120])
torch.Size([120, 1])
torch.Size([120])
torch.Size([120, 1])
torch.Size([120])
torch.Size([120, 1])
torch.Size([120])
torch.Size([120, 1])
torch.Size([120])
torch.Size([120, 1])
torch.Size([120])
torch.Size([120, 1])
torch.Size([120])
torch.Size([120, 1])
torch.Size([120])
torch.Size([120, 1])
torch.Size([120])
torch.Size([120, 1])
torch.Size([120])
torch.Size([120, 1])
torch.Size([120])
torch.Size([120, 1])
torch.Size([120])
torch.Size([120, 1])
torch.Size([120])
torch.Size([120, 1])
torch.Size([120])
torch.Size([120, 1])
torch.Size([120])
torch.Size([120, 1])
torch.Size([120])
torch.Size([120, 1])
torch.Size([120])
torch.Size([120, 1])
torch.Size([120])
torch.Size([120, 1])
torch.Size([120])
torch.Size([120, 1])
torch.Size([120])
torch.Size([120, 1])
torch.Size([120])
torch.S

In [ ]:
test_dl  = data.DataLoader(dataset_test, batch_size = 1)

In [ ]:
def test(model, valid_dl):
    model.eval()
    sum_loss = 0.0
    res = []
    target = []
    i = 0
    for elem in valid_dl:
        i += 1
        print(i)
        frames = elem[0].cuda()
        y = elem[1].float().cuda()
        y_pred = model(frames)
        res.append(y_pred.item())
        target.append(y.item())
    return res, target

In [ ]:
res = test(model, test_dl)

In [ ]:
y_pred, y = res[0], res[1]

In [ ]:
from sklearn.metrics import mean_absolute_error
pred = np.exp(y_pred)

In [ ]:
true = np.exp(y)

In [ ]:
from sklearn.metrics import mean_absolute_error

mean_absolute_error(true, pred)

In [ ]:
!pip install scipy
import scipy.stats

In [ ]:
scipy.stats.spearmanr(pred, true)